In [11]:
from itertools import product
from typing import List, Set, Tuple, Dict, Union

class SATSolver:
    def __init__(self, cnf: List[List[int]]):
        """
        Initialize the SAT Solver with a given CNF formula.
        :param cnf: List of clauses, each clause is a list of integers.
        """
        self.cnf = cnf
        self.unsatisfiable_conditions = set()  # Store unsatisfiable conditions
        self.solutions = []  # Store valid solutions

    def preprocess(self):
        """
        Sort clauses by length and group them into buckets of equal length.
        """
        self.groups = {}
        for clause in sorted(self.cnf, key=len):
            length = len(clause)
            if length not in self.groups:
                self.groups[length] = []
            self.groups[length].append(clause)

    def is_clause_satisfiable(self, clause: List[int], assignment: Dict[int, bool]) -> bool:
        """
        Check if a clause is satisfied by a given assignment.
        :param clause: Clause to be checked.
        :param assignment: Current variable assignment.
        :return: True if satisfied, False otherwise.
        """
        return any((literal > 0 and assignment.get(abs(literal), False)) or
                   (literal < 0 and not assignment.get(abs(literal), False))
                   for literal in clause)

    def solve_group(self, group: List[List[int]]) -> List[Dict[int, bool]]:
        """
        Attempt to solve a single group of clauses.
        :param group: The group of clauses to solve.
        :return: A list of satisfying assignments.
        """
        # Generate all possible assignments for variables in this group
        variables = {abs(lit) for clause in group for lit in clause}
        assignments = [dict(zip(variables, values)) for values in product([True, False], repeat=len(variables))]

        # Check each assignment
        valid_assignments = []
        for assignment in assignments:
            if all(self.is_clause_satisfiable(clause, assignment) for clause in group):
                valid_assignments.append(assignment)
        
        return valid_assignments

    def propagate_and_simplify(self, group: List[List[int]], assignments: List[Dict[int, bool]]) -> List[List[int]]:
        """
        Simplify clauses in the group based on the current assignments.
        :param group: The group of clauses to simplify.
        :param assignments: Assignments used for simplification.
        :return: Simplified group.
        """
        simplified_group = []
        for clause in group:
            reduced_clause = clause[:]
            for assignment in assignments:
                reduced_clause = [lit for lit in reduced_clause
                                  if not (lit > 0 and assignment.get(abs(lit), False)) and
                                  not (lit < 0 and not assignment.get(abs(lit), False))]
                if any((lit > 0 and assignment.get(abs(lit), False)) or
                       (lit < 0 and not assignment.get(abs(lit), False)) for lit in clause):
                    reduced_clause = []  # Clause is satisfied
                    break
            if reduced_clause:
                simplified_group.append(reduced_clause)
        return simplified_group

    def tree_based_exploration(self):
        """
        Use a tree-based approach to explore and optimize the solution space.
        """
        # Convert all groups into a tree representation
        tree = [{"clauses": self.groups[k], "assignments": []} for k in sorted(self.groups.keys())]

        for node in tree:
            # Solve each group independently
            node["assignments"] = self.solve_group(node["clauses"])
            
            # Prune unsatisfiable branches
            for assignment in node["assignments"]:
                for clause in node["clauses"]:
                    if not self.is_clause_satisfiable(clause, assignment):
                        self.unsatisfiable_conditions.add(tuple(sorted(clause)))

        # Merge equivalent nodes and remove unsatisfiable ones
        simplified_tree = []
        for node in tree:
            if node["clauses"]:
                simplified_tree.append(node)
        
        self.tree = simplified_tree

    def run(self):
        """
        Run the SAT Solver.
        """
        # Preprocess the CNF
        self.preprocess()
        
        # Iteratively solve groups and propagate solutions
        for k in sorted(self.groups.keys()):
            group = self.groups[k]
            if not group:
                continue
            assignments = self.solve_group(group)
            if not assignments:  # If no solutions are found
                return "Unsatisfiable"
            self.solutions.extend(assignments)
            self.groups[k] = self.propagate_and_simplify(group, assignments)
        
        # Use tree-based exploration for optimization
        self.tree_based_exploration()
        print(self.solutions)
        # Check if we found any valid solutions
        return "Satisfiable" if self.solutions else "Unsatisfiable"


# Example Usage
if __name__ == "__main__":
    # Example CNF formula: (x1 OR x2) AND (NOT x1 OR x3) AND (NOT x3 OR NOT x2)
    cnf =         [
            [1, 2, 3, 4, 5],
            [-1, -2, -3, -4, -5],
            [1, -2, 3, 5],
            [-2, 3,4],   # A or B
            [-2, -5],
            [-1]
        ]
    solver = SATSolver(cnf)
    result = solver.run()
    print("Result:", result)


[{1: False}, {2: True, 5: False}, {2: False, 5: True}, {2: False, 5: False}, {2: True, 3: True, 4: True}, {2: True, 3: True, 4: False}, {2: True, 3: False, 4: True}, {2: False, 3: True, 4: True}, {2: False, 3: True, 4: False}, {2: False, 3: False, 4: True}, {2: False, 3: False, 4: False}, {1: True, 2: True, 3: True, 5: True}, {1: True, 2: True, 3: True, 5: False}, {1: True, 2: True, 3: False, 5: True}, {1: True, 2: True, 3: False, 5: False}, {1: True, 2: False, 3: True, 5: True}, {1: True, 2: False, 3: True, 5: False}, {1: True, 2: False, 3: False, 5: True}, {1: True, 2: False, 3: False, 5: False}, {1: False, 2: True, 3: True, 5: True}, {1: False, 2: True, 3: True, 5: False}, {1: False, 2: True, 3: False, 5: True}, {1: False, 2: False, 3: True, 5: True}, {1: False, 2: False, 3: True, 5: False}, {1: False, 2: False, 3: False, 5: True}, {1: False, 2: False, 3: False, 5: False}, {1: True, 2: True, 3: True, 4: True, 5: False}, {1: True, 2: True, 3: True, 4: False, 5: True}, {1: True, 2: Tr

In [18]:
from typing import List, Set, Dict, Tuple, Any
import itertools
import hashlib
from collections import defaultdict

class EnhancedSATSolver:
    def __init__(self, clauses: List[List[int]]):
        """
        Initialize the Enhanced SAT Solver with a comprehensive implementation
        following the detailed description.

        :param clauses: List of clauses, where each clause is a list of literals 
                        (positive or negative integers representing variables)
        """
        # Step 1: Preprocessing - Order clauses by number of literals
        self.original_clauses = clauses
        
        # Organize clauses by number of literals
        self.clause_groups = self._organize_clauses_by_size(clauses)
        
        # Knowledge Management for Unsatisfiable Conditions
        self.unsatisfiable_conditions_table = UnsatisfiableConditionsHashTable()
        
        # Tree-based exploration data structures
        self.solution_tree = SolutionTree()
    
    def _organize_clauses_by_size(self, clauses: List[List[int]]) -> Dict[int, List[List[int]]]:
        """
        Organize clauses into groups based on their number of literals.
        
        :param clauses: Original list of clauses
        :return: Dictionary of clause groups, keyed by number of literals
        """
        clause_groups = defaultdict(list)
        for clause in clauses:
            clause_groups[len(clause)].append(clause)
        
        # Sort groups from smallest to largest
        return dict(sorted(clause_groups.items(), key=lambda x: x[0]))
    
    def solve(self) -> List[Dict[int, bool]]:
        """
        Main solving method implementing the enhanced SAT solving approach.
        
        :return: List of satisfying assignments for the entire formula
        """
        # Global list to store overall solutions
        global_solutions = []
        
        # Iterate through clause groups from smallest to largest
        for group_size, group_clauses in self.clause_groups.items():
            # Step 2: Process the smallest group first
            group_solutions = self._solve_group(group_clauses)
            
            # Check if any group is unsatisfiable
            if not group_solutions:
                return []
            
            # First group - initialize global solutions
            if not global_solutions:
                global_solutions = group_solutions
                continue
            
            # Step 3: Propagate and Simplify
            # Filter and combine solutions from previous groups
            filtered_solutions = self._filter_and_combine_solutions(
                global_solutions, 
                group_solutions, 
                group_clauses
            )
            
            # Update global solutions
            global_solutions = filtered_solutions
            
            # Stop if no solutions remain
            if not global_solutions:
                return []
        
        # Step 4: Tree-Based Optimization and Exploration
        optimized_solutions = self._optimize_solutions(global_solutions)
        
        return optimized_solutions
    
    def _solve_group(self, group_clauses: List[List[int]]) -> List[Dict[int, bool]]:
        """
        Solve a group of clauses using exhaustive search.
        
        :param group_clauses: List of clauses in the current group
        :return: List of satisfying assignments for the group
        """
        # Extract unique variables in this group
        variables = set(abs(lit) for clause in group_clauses for lit in clause)
        valid_solutions = []
        
        # Generate all possible assignments
        for assignment_values in itertools.product([True, False], repeat=len(variables)):
            # Create assignment dictionary
            assignment = dict(zip(variables, assignment_values))
            
            # Check if this assignment satisfies all clauses
            if self._is_assignment_valid(group_clauses, assignment):
                # Check against known unsatisfiable conditions
                if not self._check_unsatisfiable_conditions(assignment):
                    valid_solutions.append(assignment)
        
        return valid_solutions
    
    def _is_assignment_valid(self, clauses: List[List[int]], assignment: Dict[int, bool]) -> bool:
        """
        Check if an assignment satisfies all clauses.
        
        :param clauses: List of clauses to check
        :param assignment: Current variable assignments
        :return: True if assignment satisfies all clauses, False otherwise
        """
        for clause in clauses:
            # Clause is satisfied if at least one literal is true
            if not any(self._is_literal_true(literal, assignment) for literal in clause):
                return False
        return True
    
    def _is_literal_true(self, literal: int, assignment: Dict[int, bool]) -> bool:
        """
        Check if a specific literal is true under an assignment.
        
        :param literal: Literal to check (positive or negative integer)
        :param assignment: Current variable assignments
        :return: True if literal is satisfied, False otherwise
        """
        var = abs(literal)
        if var not in assignment:
            return False
        
        return (literal > 0 and assignment[var]) or (literal < 0 and not assignment[var])
    
    def _filter_and_combine_solutions(
        self, 
        previous_solutions: List[Dict[int, bool]], 
        new_group_solutions: List[Dict[int, bool]], 
        group_clauses: List[List[int]]
    ) -> List[Dict[int, bool]]:
        """
        Filter and combine solutions across different groups.
        
        :param previous_solutions: Solutions from previous groups
        :param new_group_solutions: Solutions from the current group
        :param group_clauses: Clauses of the current group
        :return: Filtered and combined solutions
        """
        filtered_solutions = []
        
        for prev_sol in previous_solutions:
            for new_sol in new_group_solutions:
                # Combine solutions, checking for conflicts
                combined_sol = {**prev_sol, **new_sol}
                
                # Validate the combined solution against current group
                if self._is_assignment_valid(group_clauses, combined_sol):
                    # Check against known unsatisfiable conditions
                    if not self._check_unsatisfiable_conditions(combined_sol):
                        filtered_solutions.append(combined_sol)
        
        return filtered_solutions
    
    def _check_unsatisfiable_conditions(self, assignment: Dict[int, bool]) -> bool:
        """
        Check if the current assignment matches any known unsatisfiable conditions.
        
        :param assignment: Current variable assignment
        :return: True if assignment matches an unsatisfiable condition, False otherwise
        """
        # Convert assignment to set of literals
        literal_set = set(
            lit for var, val in assignment.items() 
            for lit in [(var if val else -var)]
        )
        
        return self.unsatisfiable_conditions_table.check_condition(literal_set)
    
    def _optimize_solutions(self, solutions: List[Dict[int, bool]]) -> List[Dict[int, bool]]:
        """
        Optimize solutions through tree-based exploration and pruning.
        
        :param solutions: List of current solutions
        :return: Optimized list of solutions
        """
        optimized_solutions = []
        
        for solution in solutions:
            # Convert solution to a set of literals
            solution_literals = set(
                lit for var, val in solution.items() 
                for lit in [(var if val else -var)]
            )
            
            # Add to solution tree and get optimized representation
            optimized_solution = self.solution_tree.add_solution(solution_literals)
            
            # If solution is unique, add to optimized solutions
            if optimized_solution:
                optimized_solutions.append(solution)
            
            # Update unsatisfiable conditions if needed
            unsatisfiable_subset = self._find_unsatisfiable_subset(solution_literals)
            if unsatisfiable_subset:
                self.unsatisfiable_conditions_table.add_condition(unsatisfiable_subset)
        
        return optimized_solutions
    
    def _find_unsatisfiable_subset(self, solution_literals: Set[int]) -> Set[int]:
        """
        Find a subset of literals that leads to unsatisfiability.
        
        :param solution_literals: Set of literals in the current solution
        :return: Set of unsatisfiable literals, or empty set if not found
        """
        # Simple implementation - more sophisticated methods can be added
        for subset_size in range(1, len(solution_literals) + 1):
            for subset in itertools.combinations(solution_literals, subset_size):
                subset_set = set(subset)
                # Check if this subset leads to a contradiction
                if self._is_subset_unsatisfiable(subset_set):
                    return subset_set
        return set()
    
    def _is_subset_unsatisfiable(self, subset: Set[int]) -> bool:
        """
        Check if a subset of literals leads to a contradiction.
        
        :param subset: Set of literals to check
        :return: True if subset is unsatisfiable, False otherwise
        """
        # Basic implementation - can be extended with more sophisticated logic
        # Check for direct contradictions in the subset
        return any(-lit in subset for lit in subset)

class UnsatisfiableConditionsHashTable:
    """
    Specialized hash table for managing unsatisfiable conditions.
    """
    def __init__(self, max_entries: int = 1000):
        """
        Initialize the hash table for unsatisfiable conditions.
        
        :param max_entries: Maximum number of entries to keep
        """
        self.table = {}
        self.max_entries = max_entries
        self.entry_ages = {}
    
    def _hash_condition(self, condition: Set[int]) -> str:
        """
        Generate a hash for a set of literals.
        
        :param condition: Set of literals
        :return: Hashed string representation
        """
        # Use sorted to ensure consistent hashing
        return hashlib.md5(str(sorted(condition)).encode()).hexdigest()
    
    def add_condition(self, condition: Set[int]):
        """
        Add an unsatisfiable condition to the hash table.
        
        :param condition: Set of unsatisfiable literals
        """
        condition_hash = self._hash_condition(condition)
        
        # Manage table size
        if len(self.table) >= self.max_entries:
            # Remove oldest entry
            oldest_key = min(self.entry_ages, key=self.entry_ages.get)
            del self.table[oldest_key]
            del self.entry_ages[oldest_key]
        
        # Add new condition
        self.table[condition_hash] = condition
        self.entry_ages[condition_hash] = 0
        
        # Update ages of other entries
        for key in self.entry_ages:
            if key != condition_hash:
                self.entry_ages[key] += 1
    
    def check_condition(self, condition: Set[int]) -> bool:
        """
        Check if a condition matches any known unsatisfiable conditions.
        
        :param condition: Set of literals to check
        :return: True if condition is unsatisfiable, False otherwise
        """
        condition_hash = self._hash_condition(condition)
        return condition_hash in self.table

class SolutionTree:
    """
    Tree-based data structure for tracking and merging equivalent solutions.
    """
    def __init__(self):
        self.nodes = []
    
    def add_solution(self, solution_literals: Set[int]) -> bool:
        """
        Add a solution to the tree, merging equivalent nodes.
        
        :param solution_literals: Set of literals representing the solution
        :return: True if solution is unique, False if equivalent node exists
        """
        # Check for equivalent nodes
        for node in self.nodes:
            if self._are_nodes_equivalent(node, solution_literals):
                return False
        
        # Add new unique node
        self.nodes.append(solution_literals)
        return True
    
    def _are_nodes_equivalent(self, node1: Set[int], node2: Set[int]) -> bool:
        """
        Check if two nodes are equivalent based on their literals.
        
        :param node1: First set of literals
        :param node2: Second set of literals
        :return: True if nodes are equivalent, False otherwise
        """
        return node1 == node2

def main():
    # Example SAT problems
    test_cases = [
        # Satisfiable problem
        [
            [1, 2],        # x1 or x2
            [-1, 3],       # not x1 or x3
            [2, -3]        # x2 or not x3
        ],
        # Unsatisfiable problem
        [
            [-1],
            [1,-2]
       # not x1 or not x2
        ],
        [[1, 2], [-1, 3], [-3, -2],[-1,2]],
        [
            [1, 2, 3, 4, 5],
            [-1, -2, -3, -4, -5],
            [1, -2, 3, 5],
            [-2, 3,4],   # A or B
            [-2, -5],
        ]
    ]
    
    for i, clauses in enumerate(test_cases, 1):
        print(f"\nTest Case {i}:")
        solver = EnhancedSATSolver(clauses)
        solutions = solver.solve()
        
        if solutions:
            print("Satisfiable! Solutions:")
            for sol in solutions:
                print(sorted(sol.items()))
        else:
            print("Unsatisfiable")

if __name__ == "__main__":
    main()


Test Case 1:
Satisfiable! Solutions:
[(1, True), (2, True), (3, True)]
[(1, False), (2, True), (3, True)]
[(1, False), (2, True), (3, False)]

Test Case 2:
Satisfiable! Solutions:
[(1, True), (2, True)]
[(1, True), (2, False)]
[(1, False), (2, False)]

Test Case 3:
Satisfiable! Solutions:
[(1, False), (2, True), (3, False)]

Test Case 4:
Satisfiable! Solutions:
[(1, True), (2, True), (3, True), (4, True), (5, False)]
[(1, True), (2, True), (3, True), (4, False), (5, True)]
[(1, True), (2, True), (3, True), (4, False), (5, False)]
[(1, True), (2, True), (3, False), (4, True), (5, True)]
[(1, True), (2, True), (3, False), (4, True), (5, False)]
[(1, True), (2, True), (3, False), (4, False), (5, True)]
[(1, True), (2, True), (3, False), (4, False), (5, False)]
[(1, True), (2, False), (3, True), (4, True), (5, True)]
[(1, True), (2, False), (3, True), (4, True), (5, False)]
[(1, True), (2, False), (3, True), (4, False), (5, True)]
[(1, True), (2, False), (3, True), (4, False), (5, False)]

In [21]:
from typing import List, Set, Dict, Tuple, Any
import itertools
import hashlib
from collections import defaultdict

class EnhancedSATSolver:
    def __init__(self, clauses: List[List[int]]):
        """
        Initialize the Enhanced SAT Solver with a comprehensive implementation
        following the detailed description.

        :param clauses: List of clauses, where each clause is a list of literals 
                        (positive or negative integers representing variables)
        """
        # Preprocess clauses to extract unique variables
        self.variables = set(abs(lit) for clause in clauses for lit in clause)
        self.original_clauses = clauses
    
    def solve(self) -> List[Dict[int, bool]]:
        """
        Main solving method implementing the SAT solving approach.
        
        :return: List of satisfying assignments for the entire formula
        """
        valid_solutions = []
        
        # Generate all possible variable assignments
        for assignment_values in itertools.product([True, False], repeat=len(self.variables)):
            # Create assignment dictionary
            assignment = dict(zip(sorted(self.variables), assignment_values))
            
            # Check if this assignment satisfies all clauses
            if self._is_assignment_valid(self.original_clauses, assignment):
                valid_solutions.append(assignment)
        
        return valid_solutions
    
    def _is_assignment_valid(self, clauses: List[List[int]], assignment: Dict[int, bool]) -> bool:
        """
        Check if an assignment satisfies all clauses.
        
        :param clauses: List of clauses to check
        :param assignment: Current variable assignments
        :return: True if assignment satisfies all clauses, False otherwise
        """
        for clause in clauses:
            # Clause is satisfied if at least one literal is true
            if not any(self._is_literal_true(literal, assignment) for literal in clause):
                return False
        return True
    
    def _is_literal_true(self, literal: int, assignment: Dict[int, bool]) -> bool:
        """
        Check if a specific literal is true under an assignment.
        
        :param literal: Literal to check (positive or negative integer)
        :param assignment: Current variable assignments
        :return: True if literal is satisfied, False otherwise
        """
        var = abs(literal)
        
        # If the variable is not in the assignment, it's considered false
        if var not in assignment:
            return False
        
        # Positive literal (x) is true if assignment is True
        # Negative literal (-x) is true if assignment is False
        return (literal > 0 and assignment[var]) or (literal < 0 and not assignment[var])

def main():
    # Test cases
    test_cases = [
        # Case 1: Single-literal negative clause
        [
            [-1],     # not x1 must be true
            [1, -2]   # x1 or not x2
        ],
        # Additional test cases
        [
            [1, 2],   # x1 or x2
            [-1, 3],  # not x1 or x3
            [2, -3]   # x2 or not x3
        ],
        [
            [1, 2],   # x1 or x2
            [-1, -2]  # not x1 or not x2
        ],
        [
            [1, 2, 3, 4, 5],
            [-1, -2, -3, -4, -5],
            [1, -2, 3, 5],
            [-2, 3,4],   # A or B
            [-2, -5],
            [-1]
        ]
    ]
    
    for i, clauses in enumerate(test_cases, 1):
        print(f"\nTest Case {i}:")
        print("Clauses:", clauses)
        solver = EnhancedSATSolver(clauses)
        solutions = solver.solve()
        
        if solutions:
            print("Satisfiable! Solutions:")
            for sol in solutions:
                # Convert solution to a more readable format
                readable_sol = [(var, val) for var, val in sol.items()]
                print(readable_sol)
        else:
            print("Unsatisfiable")

if __name__ == "__main__":
    main()


Test Case 1:
Clauses: [[-1], [1, -2]]
Satisfiable! Solutions:
[(1, False), (2, False)]

Test Case 2:
Clauses: [[1, 2], [-1, 3], [2, -3]]
Satisfiable! Solutions:
[(1, True), (2, True), (3, True)]
[(1, False), (2, True), (3, True)]
[(1, False), (2, True), (3, False)]

Test Case 3:
Clauses: [[1, 2], [-1, -2]]
Satisfiable! Solutions:
[(1, True), (2, False)]
[(1, False), (2, True)]

Test Case 4:
Clauses: [[1, 2, 3, 4, 5], [-1, -2, -3, -4, -5], [1, -2, 3, 5], [-2, 3, 4], [-2, -5], [-1]]
Satisfiable! Solutions:
[(1, False), (2, True), (3, True), (4, True), (5, False)]
[(1, False), (2, True), (3, True), (4, False), (5, False)]
[(1, False), (2, False), (3, True), (4, True), (5, True)]
[(1, False), (2, False), (3, True), (4, True), (5, False)]
[(1, False), (2, False), (3, True), (4, False), (5, True)]
[(1, False), (2, False), (3, True), (4, False), (5, False)]
[(1, False), (2, False), (3, False), (4, True), (5, True)]
[(1, False), (2, False), (3, False), (4, True), (5, False)]
[(1, False), (2, 

In [24]:
from typing import List, Set, Dict, Tuple, Any
import itertools
import hashlib
from collections import defaultdict, Counter

class UnsatisfiableConditionsHashTable:
    """
    Specialized hash table for managing unsatisfiable conditions with advanced features.
    """
    def __init__(self, max_entries: int = 1000):
        """
        Initialize the hash table for unsatisfiable conditions.
        
        :param max_entries: Maximum number of entries to keep
        """
        self.table = {}
        self.max_entries = max_entries
        self.entry_ages = {}
        self.access_counter = Counter()
    
    def _hash_condition(self, condition: Set[int]) -> str:
        """
        Generate a hash for a set of literals.
        
        :param condition: Set of literals
        :return: Hashed string representation
        """
        # Use sorted to ensure consistent hashing
        return hashlib.md5(str(sorted(condition)).encode()).hexdigest()
    
    def add_condition(self, condition: Set[int]):
        """
        Add an unsatisfiable condition to the hash table.
        
        :param condition: Set of unsatisfiable literals
        """
        condition_hash = self._hash_condition(condition)
        
        # Manage table size with intelligent pruning
        if len(self.table) >= self.max_entries:
            # Remove least accessed entry
            least_accessed = min(self.access_counter, key=self.access_counter.get)
            del self.table[least_accessed]
            del self.entry_ages[least_accessed]
            del self.access_counter[least_accessed]
        
        # Add new condition
        self.table[condition_hash] = condition
        self.entry_ages[condition_hash] = 0
        self.access_counter[condition_hash] = 1
        
        # Update ages and access counts of other entries
        for key in self.entry_ages:
            if key != condition_hash:
                self.entry_ages[key] += 1
                self.access_counter[key] += 1
    
    def check_condition(self, condition: Set[int]) -> bool:
        """
        Check if a condition matches any known unsatisfiable conditions.
        
        :param condition: Set of literals to check
        :return: True if condition is unsatisfiable, False otherwise
        """
        condition_hash = self._hash_condition(condition)
        
        # Update access count if found
        if condition_hash in self.table:
            self.access_counter[condition_hash] += 1
            return True
        
        return False

class SolutionTree:
    """
    Advanced tree-based data structure for tracking and merging equivalent solutions.
    """
    def __init__(self):
        self.nodes = []
    
    def add_solution(self, solution_literals: Set[int], satisfied_clauses: Set[Tuple[int]], unsatisfied_clauses: Set[Tuple[int]]) -> bool:
        """
        Add a solution to the tree, merging equivalent nodes.
        
        :param solution_literals: Set of literals representing the solution
        :param satisfied_clauses: Set of clauses satisfied by this solution
        :param unsatisfied_clauses: Set of clauses unsatisfied by this solution
        :return: True if solution is unique, False if equivalent node exists
        """
        # Check for equivalent nodes
        for node in self.nodes:
            if self._are_nodes_equivalent(node, solution_literals, satisfied_clauses, unsatisfied_clauses):
                return False
        
        # Add new unique node
        self.nodes.append({
            'literals': solution_literals,
            'satisfied_clauses': satisfied_clauses,
            'unsatisfied_clauses': unsatisfied_clauses
        })
        return True
    
    def _are_nodes_equivalent(self, node: Dict, 
                                new_literals: Set[int], 
                                new_satisfied_clauses: Set[Tuple[int]], 
                                new_unsatisfied_clauses: Set[Tuple[int]]) -> bool:
        """
        Check if two nodes are equivalent based on their literals and clause satisfaction.
        
        :return: True if nodes are equivalent, False otherwise
        """
        return (node['satisfied_clauses'] == new_satisfied_clauses and 
                node['unsatisfied_clauses'] == new_unsatisfied_clauses)

class EnhancedSATSolver:
    def __init__(self, clauses: List[List[int]]):
        """
        Initialize the Enhanced SAT Solver with comprehensive implementation.

        :param clauses: List of clauses, where each clause is a list of literals 
                        (positive or negative integers representing variables)
        """
        # Step 1: Preprocessing - Order clauses by number of literals
        self.original_clauses = clauses
        
        # Organize clauses by number of literals
        self.clause_groups = self._organize_clauses_by_size(clauses)
        
        # Extract unique variables
        self.variables = set(abs(lit) for clause in clauses for lit in clause)
        
        # Knowledge Management for Unsatisfiable Conditions
        self.unsatisfiable_conditions_table = UnsatisfiableConditionsHashTable()
        
        # Tree-based exploration data structures
        self.solution_tree = SolutionTree()
    
    def _organize_clauses_by_size(self, clauses: List[List[int]]) -> Dict[int, List[List[int]]]:
        """
        Organize clauses into groups based on their number of literals.
        
        :param clauses: Original list of clauses
        :return: Dictionary of clause groups, keyed by number of literals
        """
        clause_groups = defaultdict(list)
        for clause in clauses:
            clause_groups[len(clause)].append(clause)
        
        # Sort groups from smallest to largest
        return dict(sorted(clause_groups.items(), key=lambda x: x[0]))
    
    def solve(self) -> List[Dict[int, bool]]:
        """
        Main solving method implementing the enhanced SAT solving approach.
        
        :return: List of satisfying assignments for the entire formula
        """
        # Global list to store overall solutions
        global_solutions = []
        
        # Iterate through clause groups from smallest to largest
        for group_size, group_clauses in self.clause_groups.items():
            # Step 2: Process the smallest group first
            group_solutions = self._solve_group(group_clauses)
            
            # Check if any group is unsatisfiable
            if not group_solutions:
                return []
            
            # First group - initialize global solutions
            if not global_solutions:
                global_solutions = group_solutions
                continue
            
            # Step 3: Propagate and Simplify
            # Filter and combine solutions from previous groups
            filtered_solutions = self._filter_and_combine_solutions(
                global_solutions, 
                group_solutions, 
                group_clauses
            )
            
            # Update global solutions
            global_solutions = filtered_solutions
            
            # Stop if no solutions remain
            if not global_solutions:
                return []
        
        # Step 4: Tree-Based Optimization and Exploration
        optimized_solutions = self._optimize_solutions(global_solutions)
        
        return optimized_solutions
    
    def _solve_group(self, group_clauses: List[List[int]]) -> List[Dict[int, bool]]:
        """
        Solve a group of clauses using exhaustive search.
        
        :param group_clauses: List of clauses in the current group
        :return: List of satisfying assignments for the group
        """
        # Extract variables in this group
        variables = set(abs(lit) for clause in group_clauses for lit in clause)
        valid_solutions = []
        
        # Generate all possible assignments
        for assignment_values in itertools.product([True, False], repeat=len(variables)):
            # Create assignment dictionary
            assignment = dict(zip(sorted(variables), assignment_values))
            
            # Check if this assignment satisfies all clauses
            if self._is_assignment_valid(group_clauses, assignment):
                # Check against known unsatisfiable conditions
                if not self._check_unsatisfiable_conditions(assignment):
                    valid_solutions.append(assignment)
        
        return valid_solutions
    
    def _is_assignment_valid(self, clauses: List[List[int]], assignment: Dict[int, bool]) -> bool:
        """
        Check if an assignment satisfies all clauses.
        
        :param clauses: List of clauses to check
        :param assignment: Current variable assignments
        :return: True if assignment satisfies all clauses, False otherwise
        """
        for clause in clauses:
            # Clause is satisfied if at least one literal is true
            if not any(self._is_literal_true(literal, assignment) for literal in clause):
                return False
        return True
    
    def _is_literal_true(self, literal: int, assignment: Dict[int, bool]) -> bool:
        """
        Check if a specific literal is true under an assignment.
        
        :param literal: Literal to check (positive or negative integer)
        :param assignment: Current variable assignments
        :return: True if literal is satisfied, False otherwise
        """
        var = abs(literal)
        
        # If the variable is not in the assignment, it's considered false
        if var not in assignment:
            return False
        
        # Positive literal (x) is true if assignment is True
        # Negative literal (-x) is true if assignment is False
        return (literal > 0 and assignment[var]) or (literal < 0 and not assignment[var])
    
    def _filter_and_combine_solutions(
        self, 
        previous_solutions: List[Dict[int, bool]], 
        new_group_solutions: List[Dict[int, bool]], 
        group_clauses: List[List[int]]
    ) -> List[Dict[int, bool]]:
        """
        Filter and combine solutions across different groups.
        
        :param previous_solutions: Solutions from previous groups
        :param new_group_solutions: Solutions from the current group
        :param group_clauses: Clauses of the current group
        :return: Filtered and combined solutions
        """
        filtered_solutions = []
        
        for prev_sol in previous_solutions:
            for new_sol in new_group_solutions:
                # Combine solutions, checking for conflicts
                combined_sol = {**prev_sol, **new_sol}
                
                # Validate the combined solution against current group
                if self._is_assignment_valid(group_clauses, combined_sol):
                    # Check against known unsatisfiable conditions
                    if not self._check_unsatisfiable_conditions(combined_sol):
                        filtered_solutions.append(combined_sol)
        
        return filtered_solutions
    
    def _check_unsatisfiable_conditions(self, assignment: Dict[int, bool]) -> bool:
        """
        Check if the current assignment matches any known unsatisfiable conditions.
        
        :param assignment: Current variable assignment
        :return: True if assignment matches an unsatisfiable condition, False otherwise
        """
        # Convert assignment to set of literals
        literal_set = set(
            lit for var, val in assignment.items() 
            for lit in [(var if val else -var)]
        )
        
        return self.unsatisfiable_conditions_table.check_condition(literal_set)
    
    def _optimize_solutions(self, solutions: List[Dict[int, bool]]) -> List[Dict[int, bool]]:
        """
        Optimize solutions through tree-based exploration and pruning.
        
        :param solutions: List of current solutions
        :return: Optimized list of solutions
        """
        optimized_solutions = []
        
        for solution in solutions:
            # Convert solution to a set of literals
            solution_literals = set(
                lit for var, val in solution.items() 
                for lit in [(var if val else -var)]
            )
            
            # Identify satisfied and unsatisfied clauses
            satisfied_clauses = set(
                tuple(clause) for clause in self.original_clauses 
                if self._is_assignment_valid([clause], solution)
            )
            
            unsatisfied_clauses = set(
                tuple(clause) for clause in self.original_clauses 
                if not self._is_assignment_valid([clause], solution)
            )
            
            # Add to solution tree and get optimized representation
            is_unique = self.solution_tree.add_solution(
                solution_literals, 
                satisfied_clauses, 
                unsatisfied_clauses
            )
            
            # If solution is unique, add to optimized solutions
            if is_unique:
                optimized_solutions.append(solution)
            
            # Update unsatisfiable conditions if needed
            unsatisfiable_subset = self._find_unsatisfiable_subset(solution_literals)
            if unsatisfiable_subset:
                self.unsatisfiable_conditions_table.add_condition(unsatisfiable_subset)
        
        return optimized_solutions
    
    def _find_unsatisfiable_subset(self, solution_literals: Set[int]) -> Set[int]:
        """
        Find a subset of literals that leads to unsatisfiability.
        
        :param solution_literals: Set of literals in the current solution
        :return: Set of unsatisfiable literals, or empty set if not found
        """
        # Systematic approach to find unsatisfiable subsets
        for subset_size in range(1, len(solution_literals) + 1):
            for subset in itertools.combinations(solution_literals, subset_size):
                subset_set = set(subset)
                # Check if this subset leads to a contradiction
                if self._is_subset_unsatisfiable(subset_set):
                    return subset_set
        return set()
    
    def _is_subset_unsatisfiable(self, subset: Set[int]) -> bool:
        """
        Check if a subset of literals leads to a contradiction.
        
        :param subset: Set of literals to check
        :return: True if subset is unsatisfiable, False otherwise
        """
        # Check for direct contradictions in the subset
        return any(-lit in subset for lit in subset)

def main():
    # Test cases
    test_cases = [
        # Case 1: Single-literal negative clause
        [
            [-1],     # not x1 must be true
            [1, -2]   # x1 or not x2
        ],
        # Case 2: Satisfiable problem
        [
            [1, 2],   # x1 or x2
            [-1, 3],  # not x1 or x3
            [2, -3]   # x2 or not x3
        ],
        # Case 3: Unsatisfiable problem
        [
            [1, 2],   # x1 or x2
            [-1, -2],
            [-3,1]
            ,[3]  # not x1 or not x2
        ]
    ]
    
    for i, clauses in enumerate(test_cases, 1):
        print(f"\nTest Case {i}:")
        print("Clauses:", clauses)
        solver = EnhancedSATSolver(clauses)
        solutions = solver.solve()
        
        if solutions:
            print("Satisfiable! Solutions:")
            for sol in solutions:
                # Convert solution to a more readable format
                readable_sol = [(var, val) for var, val in sol.items()]
                print(readable_sol)
        else:
            print("Unsatisfiable")

if __name__ == "__main__":
    main()


Test Case 1:
Clauses: [[-1], [1, -2]]
Satisfiable! Solutions:
[(1, True), (2, True)]
[(1, False), (2, False)]

Test Case 2:
Clauses: [[1, 2], [-1, 3], [2, -3]]
Satisfiable! Solutions:
[(1, True), (2, True), (3, True)]

Test Case 3:
Clauses: [[1, 2], [-1, -2], [-3, 1], [3]]
Satisfiable! Solutions:
[(3, True), (1, True), (2, False)]
[(3, False), (1, True), (2, False)]
